In [8]:
import time
import pandas as pd
import numpy as np
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import requests
import re

#DF선언
saramIn_df = pd.DataFrame()
saramIn_df_temp = pd.DataFrame()
saramIn_df_temp = saramIn_df_temp.append(
                    {"coID":"",#c
                     "coIndx":"",#c
                     "coNm":"",#c
                     "city":"",#c
                     "cond_a":"",#c
                     "cond_b":"",#c
                     "cond_c":"",#c
                     "title":"", #c
                     "region":"",#c
                     "r_date":""
                     #"view_cnt":"",
                     #"scrap_cnt":"",
                     #"apply_cnt":"",
                     #"20_apply":"",
                     #"30_apply":"",
                     #"40_apply":"",
                     #"male_apply_cnt":"",
                     #"female_apply_cnt":"",
                     #"y_income":""
                        
                    },ignore_index=True

)

#URL 가져오기!!
driver = webdriver.Chrome("C:\\dev\\chromedriver.exe")

driver.get("http://www.saramin.co.kr/zf_user/search/recruit?searchType=search&searchword="+"인공지능"+"&loc_mcd=115000%2C114000%2C116000&company_cd=0,1,2,3,4,5,6,7,9&panel_type=&search_optional_item=y&search_done=y&panel_count=y&recruitPage=2&recruitSort=relation&recruitPageCount=50&inner_com_type=&quick_apply=")
html = driver.page_source
soup = bs(html, "html.parser")


#불러올 데이터 총 개수 찾기!
cnt_result = soup.find("span", class_="cnt_result").text
cnt_result = re.sub(r"총 |건","",cnt_result)
cnt_result = cnt_result.split(",")
totCnt =""
for i in range(len(cnt_result)):
    totCnt += cnt_result[i]
totCnt = int(totCnt)
#불러올 데이터 수
totCnt
#불러올 페이지수
totPage = totCnt//50+1
#print(totPage)

#페이지 돌리기(totPage)
for page in range(1,totPage+1):
    driver.get("http://www.saramin.co.kr/zf_user/search/recruit?searchType=search&searchword="+"인공지능"+"&loc_mcd=115000%2C114000%2C116000&company_cd=0,1,2,3,4,5,6,7,9&panel_type=&search_optional_item=y&search_done=y&panel_count=y&recruitPage="+str(page)+"&recruitSort=relation&recruitPageCount=50&inner_com_type=&quick_apply=")
    p_html = driver.page_source
    p_soup = bs(html, "html.parser")
    
    comp_list =p_soup.find_all("div", class_="item_recruit")
    
    for view in range(len(comp_list)):#len(comp_list)
        #column 값들 가져오기!
        coID = comp_list[view].find("div",class_="area_corp_info").find("a").get("href")
        coID = re.findall(r"[0-9]{10}",coID)[0]
        coIndx = comp_list[view].find("h2",class_="job_tit").find("a").get("href")
        coIndx = re.findall(r"[0-9]{8}",coIndx)[0]
        coNm = comp_list[view].find("div", class_="area_corp").strong.a.get("title")
        city = comp_list[view].find("div", class_="job_condition").find_all("span")[0].find_all("a")[0].text
        if len(comp_list[view].find("div", class_="job_condition").find_all("span")[0].find_all("a")) == 2:
            region = comp_list[view].find("div", class_="job_condition").find_all("span")[0].find_all("a")[1].text
        else:
            region =""
        
        if len(comp_list[view].find("div", class_="job_condition").find_all("span"))>= 4:
            cond_a = comp_list[view].find("div", class_="job_condition").find_all("span")[1].text
            cond_a = re.sub(r"↑","",cond_a)
            cond_a = re.sub(r"·","",cond_a)
            cond_a = re.sub(r" ","",cond_a)
            cond_b = comp_list[view].find("div", class_="job_condition").find_all("span")[2].text
            cond_b = re.sub(r"↑","",cond_b)
            cond_c = comp_list[view].find("div", class_="job_condition").find_all("span")[3].text
            cond_c = re.sub(r"↑","",cond_c)
            cond_c = re.sub(r"·","",cond_c)
            cond_c = re.sub(r" ","",cond_c)
            
            
            
            #cond_c

        elif len(comp_list[view].find("div", class_="job_condition").find_all("span"))< 4:
            cond_a =  comp_list[view].find("div", class_="job_condition").find_all("span")[1].text
            cond_a = re.sub(r"↑","",cond_a)
            cond_a = re.sub(r"·","",cond_a)
            cond_a = re.sub(r" ","",cond_a)
            cond_b = comp_list[view].find("div", class_="job_condition").find_all("span")[2].text
            cond_b = re.sub(r"↑","",cond_b)
            cond_c = ""
        
        title = comp_list[view].find("h2").find("a").get("title")
        title = re.sub(r" ","",title)
        r_date = comp_list[view].find("span",class_="date").text
        
        #print(coIndx)
        #드라이버종료
        #정보추가얻기 #드라이버 재시작
        
        saramIn_df_temp.iat[0,saramIn_df_temp.columns.get_loc("coID")] =coID
        saramIn_df_temp.iat[0,saramIn_df_temp.columns.get_loc("coIndx")] =coIndx
        saramIn_df_temp.iat[0,saramIn_df_temp.columns.get_loc("coNm")] =coNm
        saramIn_df_temp.iat[0,saramIn_df_temp.columns.get_loc("city")] =city
        saramIn_df_temp.iat[0,saramIn_df_temp.columns.get_loc("cond_a")] =cond_a
        saramIn_df_temp.iat[0,saramIn_df_temp.columns.get_loc("cond_b")] =cond_b
        saramIn_df_temp.iat[0,saramIn_df_temp.columns.get_loc("cond_c")] =cond_c
        saramIn_df_temp.iat[0,saramIn_df_temp.columns.get_loc("title")] =title
        saramIn_df_temp.iat[0,saramIn_df_temp.columns.get_loc("region")] =region
        saramIn_df_temp.iat[0,saramIn_df_temp.columns.get_loc("r_date")] = r_date
        saramIn_df = saramIn_df.append(saramIn_df_temp,ignore_index=True)
        
        saramIn_df_temp.drop(0)
        
        
        
    

saramIn_df
saramIn_df.to_csv("saramIn_df_bigData_02.csv")





AttributeError: 'NoneType' object has no attribute 'text'

In [46]:
saramIn_df.to_csv("saramIn_df_dataAnalyist_etc.csv")

In [43]:
import time
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
from selenium import webdriver


driver = webdriver.Chrome("chromedriver.exe")
saramIn_df2 = pd.DataFrame()
saramIn_df2_temp = pd.DataFrame()


saramIn_df2_temp = saramIn_df2_temp.append(
                    {"coIndx":"", 
                     "view_cnt":"",
                     "scrap_cnt":""
                     #"apply_cnt":"",
                     #"20_apply":"",
                     #"30_apply":"",
                     #"40_apply":"",
                     #"male_apply_cnt":"",
                     #"female_apply_cnt":"",
                     #"y_income":""
                        
                    },ignore_index=True

)
#driver = webdriver.Chrome("C:\\dev\\chromedriver.exe")

for i in (saramIn_df["coIndx"]):
    URL = "http://www.saramin.co.kr/zf_user/jobs/relay/view?isMypage=no&rec_idx="+i+"&recommend_ids=eJxdkMkVwyAMBavJXWj%2FZxdC%2F10EJwiwj%2FNG%2BloklVmSeqh%2F4pJUkkjrBv2hIA3czdsL%2B%2FSRqodfWN7IQDtuYXmECa%2FpG8s7lLI7ajtTs2PZwlk%2BjgnVkW8vnL6FRMPuZ8qRvssLV1xztMMXHp7s4f84PSvGS7oTz2PGfDlQw30jBQdvjCaB9bk7Gkcvwe9HQ79ynWMH&view_type=search&searchword=데이터분석&searchType=default_mysearch&gz=1&t_ref_content=generic&t_ref=search&paid_fl=n#seq=0"
    print(URL)
    driver.get(URL)

    #i
    scrap_cnt =driver.find_element_by_css_selector("#content > div.wrap_jview > div.jview.jview-0-"+i+" > div.wrap_jv_cont > div.wrap_jv_header > div >button.btn_scrap.scrap-"+i+" > span").text
    view_cnt = driver.find_element_by_css_selector("#content > div.wrap_jview > div.jview.jview-0-"+i+" > div.wrap_jv_cont > div.jv_cont.jv_summary > div.cont > ul > li:nth-child(1) > strong").text
    #apply_cnt = driver.find_element_by_css_selector("#content > div.wrap_jview > div.jview.jview-0-"+i+" > div.wrap_jv_cont > div.jv_cont.jv_statics > div.cont > div > div > div.box_chart.narrow.count.sub > dl > dd > span").text
    coIndx = i
    
    saramIn_df2_temp.iat[0,saramIn_df2_temp.columns.get_loc("coIndx")] =coIndx
    saramIn_df2_temp.iat[0,saramIn_df2_temp.columns.get_loc("view_cnt")] =view_cnt
    saramIn_df2_temp.iat[0,saramIn_df2_temp.columns.get_loc("scrap_cnt")] =scrap_cnt
    
    saramIn_df2 = saramIn_df2.append(saramIn_df2_temp,ignore_index=True)
    
    saramIn_df2_temp.drop(0)
    
    
    
   
    driver.implicitly_wait(0.3)
    

        
#URL_page = requests.get(URL)
#soup = bs(URL_page.content,"html.parser")
#print(soup.select("span.txt_scrap"))
    
    

    


http://www.saramin.co.kr/zf_user/jobs/relay/view?isMypage=no&rec_idx=38262896&recommend_ids=eJxdkMkVwyAMBavJXWj%2FZxdC%2F10EJwiwj%2FNG%2BloklVmSeqh%2F4pJUkkjrBv2hIA3czdsL%2B%2FSRqodfWN7IQDtuYXmECa%2FpG8s7lLI7ajtTs2PZwlk%2BjgnVkW8vnL6FRMPuZ8qRvssLV1xztMMXHp7s4f84PSvGS7oTz2PGfDlQw30jBQdvjCaB9bk7Gkcvwe9HQ79ynWMH&view_type=search&searchword=데이터분석&searchType=default_mysearch&gz=1&t_ref_content=generic&t_ref=search&paid_fl=n#seq=0
http://www.saramin.co.kr/zf_user/jobs/relay/view?isMypage=no&rec_idx=38251713&recommend_ids=eJxdkMkVwyAMBavJXWj%2FZxdC%2F10EJwiwj%2FNG%2BloklVmSeqh%2F4pJUkkjrBv2hIA3czdsL%2B%2FSRqodfWN7IQDtuYXmECa%2FpG8s7lLI7ajtTs2PZwlk%2BjgnVkW8vnL6FRMPuZ8qRvssLV1xztMMXHp7s4f84PSvGS7oTz2PGfDlQw30jBQdvjCaB9bk7Gkcvwe9HQ79ynWMH&view_type=search&searchword=데이터분석&searchType=default_mysearch&gz=1&t_ref_content=generic&t_ref=search&paid_fl=n#seq=0
http://www.saramin.co.kr/zf_user/jobs/relay/view?isMypage=no&rec_idx=38399122&recommend_ids=eJxdkMkVwyAMBavJXWj%2FZxdC%2F10EJwiwj%2FNG%2BloklV

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#content > div.wrap_jview > div.jview.jview-0-38284671 > div.wrap_jv_cont > div.wrap_jv_header > div >button.btn_scrap.scrap-38284671 > span"}
  (Session info: chrome=83.0.4103.116)


In [29]:
import time
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
from selenium import webdriver

driver = webdriver.Chrome("chromedriver.exe")
saramIn_df2 = pd.DataFrame()
saramIn_df2_temp = pd.DataFrame()
URL = "http://www.saramin.co.kr/zf_user/jobs/relay/view?isMypage=no&rec_idx="+"38422380"+"&recommend_ids=eJxdkMkVwyAMBavJXWj%2FZxdC%2F10EJwiwj%2FNG%2BloklVmSeqh%2F4pJUkkjrBv2hIA3czdsL%2B%2FSRqodfWN7IQDtuYXmECa%2FpG8s7lLI7ajtTs2PZwlk%2BjgnVkW8vnL6FRMPuZ8qRvssLV1xztMMXHp7s4f84PSvGS7oTz2PGfDlQw30jBQdvjCaB9bk7Gkcvwe9HQ79ynWMH&view_type=search&searchword=데이터분석&searchType=default_mysearch&gz=1&t_ref_content=generic&t_ref=search&paid_fl=n#seq=0"
print(URL)
driver.get(URL)
driver.find_element_by_css_selector("#content > div.wrap_jview > div.jview.jview-1-38422380 > div.wrap_jv_cont > div.jv_cont.jv_statics > div.cont > div > div > div.box_chart.narrow.count.sub > dl > dd > span").text


http://www.saramin.co.kr/zf_user/jobs/relay/view?isMypage=no&rec_idx=38422380&recommend_ids=eJxdkMkVwyAMBavJXWj%2FZxdC%2F10EJwiwj%2FNG%2BloklVmSeqh%2F4pJUkkjrBv2hIA3czdsL%2B%2FSRqodfWN7IQDtuYXmECa%2FpG8s7lLI7ajtTs2PZwlk%2BjgnVkW8vnL6FRMPuZ8qRvssLV1xztMMXHp7s4f84PSvGS7oTz2PGfDlQw30jBQdvjCaB9bk7Gkcvwe9HQ79ynWMH&view_type=search&searchword=데이터분석&searchType=default_mysearch&gz=1&t_ref_content=generic&t_ref=search&paid_fl=n#seq=0


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#content > div.wrap_jview > div.jview.jview-1-38422380 > div.wrap_jv_cont > div.jv_cont.jv_statics > div.cont > div > div > div.box_chart.narrow.count.sub > dl > dd > span"}
  (Session info: chrome=83.0.4103.116)


In [ ]:
 sec_URL = "http://www.saramin.co.kr/zf_user/jobs/relay/view?isMypage=no&rec_idx="+coIndx+"&recommend_ids=eJxdkMsZwzAIg6fpHQMCc%2B4g3n%2BLkjqGfDn%2BSLwkU3yq8nL1j3%2FlgUv%2FBaUsYSE2DhcfsWBj22MieFnMF57uAda0B7YOQlANa7ztEg65ttsLjz4BkWywF9axUKD6G%2BucAFGd01i6WC5s%2FeCt82Byy%2F%2F2%2FHyN2CqMYeBnNg7O7QfTnKVlxFu1jFIaydm50Yd4dFAUptch%2BgMgbV9P&view_type=search&searchword=데이터분석&searchType=default_mysearch&gz=1&t_ref_content=generic&t_ref=search&paid_fl=n#seq=0"
        print(sec_URL)
        sec_html = driver.get(sec_URL)
        sec_html = driver.page_source
        
        soup = bs(sec_html, "html.parser")
        
        #if soup.find_all("span",class_="txt_scrap")!=[]:
        scrap_cnt = soup.find_all("span",class_="txt_scrap")[0].text
        print(scrap_cnt)
        #else:
            #scrap_cnt = soup.find_all("span",class_="txt_scrap")
            #print(scrap_cnt)
        
        #초기화
        #sec_URL=""
        #sec_html=""
        #soup=""